# Simulating a retina #

In [1]:
%matplotlib inline
import numpy as np
import cv2 as cv2
from time import time

First, let us grab a stream from the laptop camera and display it. We're making the data greyscale because we are not modelling the colour properties of the retina here. (If this crashes, it is likely you will need to specify a different number for `video_src` below.)

In [2]:
# initialise the video capture
video_src = 0                       # inbuilt camera <= try video_src = 1 if this crashes.
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64') / 255

    # display
    disp = img
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

Next, we add a step to model the logarithmic response of our photoreceptors.

In [3]:
## initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = np.zeros((height, width))

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0)), subtract 4 for display purpose only
    img = np.log(img + 1) - 4
#########################

    # display
    disp = img
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

We then add a high-pass filter, by subtracting a low-pass filtered version of the stream from each input frame. This models the sensitivity to change of the Centre pathway. 

In [4]:
# initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img
###############################################

    # display
    disp = img_hp
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

The pure high-pass filter above has zero gain at DC. This is not the case in the retina, where the response to DC is reduced but not exactly zero. To model this, we reduce the low-pass filters signal by a factor `w_u` before subraction.

In [5]:
# initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - w_u * img_lp
#                  ############
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img

    # display
    disp = img_hp
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

The response in the Centre pathway is actually a temporal band-pass filter, which we can create by low-pass filtering the high-pass response above with a different (shorter) time-constant. We apply this low-pass filter twice in succession. Note `tau_u` and `tau_c` below for the high-pass and low-pass time constants, respectively.

In [6]:
# initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
centre0 = img_lp.copy()
centre = img_lp.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0
#####################################################

    # display
    disp = centre
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

Now, lets start creating the surround response. It consists of a spatially AND temporally low-pass filtered version of the Centre signal.

In [7]:
# initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
centre0 = img_lp.copy()
centre = img_lp.copy()
surround = img_lp.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0

    # spatial filter image
    surroundnew = cv2.GaussianBlur(centre, (0, 0), sigma)
#########################################################

    # low pass filter surround
    surround = (1 - LPF_s) * surround + LPF_s * surroundnew
#########################################

    # display
    disp = surround
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

The bipolar cells in the retina are driven by the difference between the Centre and the Surround signal.

In [10]:
# initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
centre0 = img_lp.copy()
centre = img_lp.copy()
surround = img_lp.copy()
vbip = img_lp.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)
tau_a = 0.01
LPF_a = 1 / (fps * tau_a + 1)

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0

    # spatial filter image
    surroundnew = cv2.GaussianBlur(centre, (0, 0), sigma)

    # low pass filter surround
    surround = (1 - LPF_s) * surround + LPF_s * surroundnew

    # calculate difference between centre and surround
    iopl = centre - surround
############################

    # low pass filter to create the bipolar cell activation
    vbip = (1 - LPF_a) * vbip + LPF_a * iopl
############################################

    # display
    disp = vbip + 0.5 # add 0.5 to make 0 grey
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

In [15]:
### initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
centre0 = img_lp.copy()
centre = img_lp.copy()
surround = img_lp.copy()
vbip = img_lp.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)
tau_a = 0.01
LPF_a = 1 / (fps * tau_a + 1)
l_a = 0.2

last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img = np.log(img + 1)

    # high pass filter centre
    img_hp = img - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0

    # spatial filter image
    surroundnew = cv2.GaussianBlur(centre, (0, 0), sigma)

    # low pass filter surround
    surround = (1 - LPF_s) * surround + LPF_s * surroundnew

    # calculate difference between centre and surround
    iopl = centre - surround

    # low pass filter to create the bipolar cell activation
    vbip = (1 - LPF_a) * vbip + LPF_a * iopl
    
    # automatic gain control from amacrine cells
    ga = l_a * cv2.GaussianBlur(vbip**2, (0, 0), sigma)
    
    # update bipolar activation with gain control via shunting inhibition
    vbip = (1 - ga) * vbip

    # display
    disp = vbip + 0.5 # add 0.5 to make 0 grey
    cv2.putText(disp, "FPS = %f"%rate, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break

# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

Now, let's display them all together.

In [64]:
##### initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
img_log = img_lp.copy()
img_hp = img_lp.copy()
centre0 = img_lp.copy()
centre = img_lp.copy()
surround = img_lp.copy()
vbip = img_lp.copy()
ref = np.log(img_lp + 1)
evt = ref.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)
tau_a = 0.05
LPF_a = 1 / (fps * tau_a + 1)
l_a = 10
eth = 0.1

pause = False
last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img_log = np.log(img + 1)

    # high pass filter centre
    img_hp = img_log - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img_log

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0

    # spatial filter image
    surroundnew = cv2.GaussianBlur(centre, (0, 0), sigma)

    # low pass filter surround
    surround = (1 - LPF_s) * surround + LPF_s * surroundnew

    # calculate difference between centre and surround
    iopl = (centre - surround) * 2

    # low pass filter to create the bipolar cell activation
    vbip = (1 - LPF_a) * vbip + LPF_a * iopl
    
    # automatic gain control from amacrine cells
    ga = l_a * cv2.GaussianBlur(vbip**2, (0, 0), sigma)
    # ga = 0
    
    # update ganglion activation with gain control via shunting inhibition
    IPL = (1 - ga) * vbip
    
    # On and Off ganglion cell spike rate
    gon = IPL * (IPL > 0) * 2
    goff = -IPL * (IPL < 0) * 2
    
    # DVS algorithm below
    dif = img_log - ref

    # detect on and off events
    on = dif > eth
    off = -dif > eth

    # update reference image where events have happened
    ref[on] = img_log[on]
    ref[off] = img_log[off]

    # create image
    evt = (0.5 - 0.5 * off)
    evt += (0.5 * on)

    if not pause:
        # display
        disp = np.zeros((3 * height, 3 * width))
        disp = np.vstack([np.hstack([img / 255, surround, centre]), 
                          np.hstack([iopl, vbip, IPL]) + 0.5,
                          np.hstack([gon, goff, evt])
                         ])
        
        cv2.putText(disp, "FPS = %f"%rate, (10, 3 * height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Input", (1 * width // 2 - 10, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 0)
        cv2.putText(disp, "Surround", (3 * width // 2 - 15, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Centre", (5 * width // 2 - 15, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "OPL", (1 * width // 2 - 10, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Bipolar", (3 * width // 2 - 10, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "IPL", (5 * width // 2 - 10, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "On", (1 * width // 2 - 10, 2 * height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Off", (3 * width // 2 - 10, 2 * height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "DVS", (5 * width // 2 - 10, 2 * height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break
    if c == ord(" "):
        pause = not pause
        
# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

In [2]:
##### initialise the video capture
video_src = 0                       # inbuilt camera
cam = cv2.VideoCapture(video_src)   # activate the camera
ret, img = cam.read()               # get one default frame

# set image size to VGA to reduce computational effort
height = 480                        # desired height
width = 640                         # desired width
native_width = cam.get(3)           # get native frame width
native_height = cam.get(4)          # get native frame height
fps = cam.get(5)                    # get default frame rate
ret = cam.set(3, width)             # set frame height            
ret = cam.set(4, height)            # set frame width
ret, img = cam.read()               # get one frame of set size

# initialise first filtered frames
img_lp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')
img_log = img_lp.copy()
img_hp = img_lp.copy()
centre0 = img_lp.copy()
centre = img_lp.copy()
surround = img_lp.copy()
vbip = img_lp.copy()
ref = np.log(img_lp + 1)
evt = ref.copy()

# initialise the windows and their positions
cv2.namedWindow("Event Based Camera | q: quit")

sigma = 5
w_u = 0.9
tau_u = 0.1
HPF_c = 1 / (fps * tau_u + 1)
tau_c = 0.01
LPF_c = 1 / (fps * tau_c + 1)
tau_s = 0.1
LPF_s = 1 / (fps * tau_s + 1)
tau_a = 0.05
LPF_a = 1 / (fps * tau_a + 1)
l_a = 10
eth = 0.1

pause = False
last_t = time()
while True:
    # get a frame
    ret, img = cam.read()

    # calculate frame rate
    t = time()
    rate = 1.0/(t-last_t)
    last_t = t

    # make image greyscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    # logarithmic compression (log(I + 1.0))
    img_log = np.log(img + 1)

    # high pass filter centre
    img_hp = img_log - w_u * img_lp
    img_lp = (1 - HPF_c) * img_lp + HPF_c * img_log

    # low pass filter centre
    centre0 = (1 - LPF_c) * centre0 + LPF_c * img_hp
    centre  = (1 - LPF_c) * centre  + LPF_c * centre0

    # spatial filter image
    surroundnew = cv2.GaussianBlur(centre, (0, 0), sigma)

    # low pass filter surround
    surround = (1 - LPF_s) * surround + LPF_s * surroundnew

    # calculate difference between centre and surround
    iopl = (centre - surround) * 2

    # low pass filter to create the bipolar cell activation
    vbip = (1 - LPF_a) * vbip + LPF_a * iopl
    
    # automatic gain control from amacrine cells
    ga = l_a * cv2.GaussianBlur(vbip**2, (0, 0), sigma)
    # ga = 0
    
    # update ganglion activation with gain control via shunting inhibition
    IPL = (1 - ga) * vbip
    
    # On and Off ganglion cell spike rate
    gon = IPL * (IPL > 0) + 0.5 
    goff = -IPL * (IPL < 0) + 0.5
    
    # DVS algorithm below
    dif = img_log - ref

    # detect on and off events
    on = dif > eth
    off = -dif > eth

    # update reference image where events have happened
    ref[on] = img_log[on]
    ref[off] = img_log[off]

    # create image
    evt = (0.5 - 0.5 * off)
    evt += (0.5 * on)

    if not pause:
        # display
        disp = np.zeros((2 * height, 3 * width))
        disp = np.vstack([np.hstack([img / 255, centre, IPL + 0.5]), 
                          np.hstack([gon, goff, evt])
                         ])
        
        cv2.putText(disp, "FPS = %f"%rate, (10, 2 * height - 10), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Input",   (1 * width // 2 - 10, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 0)
        cv2.putText(disp, "Centre",  (3 * width // 2 - 15, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Bipolar", (5 * width // 2 - 15, 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "On Ganglion" , (1 * width // 2 - 15, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "Off Ganglion", (3 * width // 2 - 15, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.putText(disp, "DVS"         , (5 * width // 2 - 10, height + 20), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
        cv2.imshow("Event Based Camera | q: quit", disp)

    # monitor the keyboard
    c = cv2.waitKey(1) & 0xFF            
    # for 'q', quit
    if c == ord("q"):
        break
    if c == ord(" "):
        pause = not pause
        
# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF

In [32]:
# clean up for exit
cam.release()  
cv2.destroyAllWindows()
c = cv2.waitKey(1) & 0xFF